In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd drive/My \Drive/NLP

In [ ]:
!pip install textattack==0.3.7

In [ ]:
!pip install lime shap

In [ ]:
# Load libraries
import pandas as pd
import numpy as np
import nltk
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
from lime.lime_text import LimeTextExplainer
from transformers import BertTokenizer
import random
import transformers
import textattack
import shap
from textattack.datasets import Dataset
from datasets import load_dataset
from transformers import GPT2Tokenizer
from textattack.models.wrappers import ModelWrapper
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack import Attack
from textattack.transformations import WordSwapEmbedding
from textattack.search_methods import GreedyWordSwapWIR
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.constraints.semantics import WordEmbeddingDistance
from lime.lime_text import LimeTextExplainer
from nltk.corpus import wordnet

nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

In [ ]:
# Load the dataset
df = pd.read_csv('./Data/KaggleData.csv')

# Preprocess the tweets
df = df.dropna(subset=['tweet', 'class'])
df = df.reset_index(drop=True)

# Convert to lowercase, remove punctuation, extra spaces, URLs, mentions, and hashtags
df['tweet'] = df['tweet'].str.lower().replace(r'[^\w\s]', '', regex=True).replace(' {2,}', ' ', regex=True).replace('"', '')
df['tweet'] = df['tweet'].replace(r'http\S+|www.\S+|@\w+|#\w+', '', regex=True)

# Download NLTK resources
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

# Tokenization
df['tweet'] = df['tweet'].apply(nltk.word_tokenize)

# Lemmatization
lemmatizer = nltk.WordNetLemmatizer()
df['tweet'] = df['tweet'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

# Removing stop-words
stop_words = set(nltk.corpus.stopwords.words('english'))
df['tweet'] = df['tweet'].apply(lambda x: [word for word in x if word not in stop_words])

# Prepare data for DataLoader
max_length = 50
df['tweet'] = df['tweet'].apply(lambda x: x[:max_length])

# Create word2index dictionary
word2index = {}
for tweet in df['tweet']:
    for word in tweet:
        if word not in word2index:
            word2index[word] = len(word2index)

# Convert words to indices and pad sequences
input_data = np.zeros((len(df['tweet']), max_length), dtype=int)
for i, tweet in enumerate(df['tweet']):
    for j, word in enumerate(tweet):
        input_data[i, j] = word2index[word]

# Encode labels
# 0 - hate speech, 1 - offensive language, 2 - neither
le = LabelEncoder()
y = le.fit_transform(df['class'])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(input_data, y, test_size=0.3, stratify=y, random_state=42)

# Create DataLoader objects
batch_size = 64
train_dataset = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
test_dataset = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Create CNN model
class TextCNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes):
        super(TextCNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.conv1 = nn.Conv2d(1, 100, (3, embed_dim))
        self.conv2 = nn.Conv2d(1, 100, (4, embed_dim))
        self.conv3 = nn.Conv2d(1, 100, (5, embed_dim))
        self.fc = nn.Linear(300, num_classes)

    def forward(self, x):
        x = self.embedding(x).unsqueeze(1)
        x1 = F.relu(self.conv1(x)).squeeze(3)
        x2 = F.relu(self.conv2(x)).squeeze(3)
        x3 = F.relu(self.conv3(x)).squeeze(3)
        x1 = F.max_pool1d(x1, x1.size(2)).squeeze(2)
        x2 = F.max_pool1d(x2, x2.size(2)).squeeze(2)
        x3 = F.max_pool1d(x3, x3.size(2)).squeeze(2)
        x = torch.cat((x1, x2, x3), 1)
        x = self.fc(x)
        return x

# Training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vocab_size = len(word2index)
embed_dim = 100
num_classes = len(np.unique(y))
model = TextCNN(vocab_size, embed_dim, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs.long())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")

    # Save Model
    torch.save(model, './Weights/KaggleCNN_SHAP.pth')

model = torch.load('./Weights/KaggleCNN_SHAP.pth')

# Test the model and collect predictions and true labels
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs.long())
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Calculate accuracy, precision, recall, F1-score, and confusion matrix
accuracy = np.mean(np.array(predictions) == np.array(true_labels))
precision, recall, f1_score, _ = precision_recall_fscore_support(true_labels, predictions, average='weighted')
conf_mat = confusion_matrix(true_labels, predictions)

print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1-score: ", f1_score)
print("Confusion Matrix:\n", conf_mat)

In [ ]:
# Wrapper for the TextCNN model
class CNNBaselineWrapper:
    def __init__(self, model):
        self.model = model

    def __call__(self, text_input_list):
        preds = []
        for text in text_input_list:
            input_tensor = tokenize_and_pad([text]).long()
            output = self.model(input_tensor.to(device))
            pred = torch.softmax(output, dim=1).squeeze().tolist()
            preds.append(pred)
        return np.array(preds)

def tokenize_and_pad(text_list):
    max_length = 50
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    tokens = [tokenizer.tokenize(text)[:max_length] for text in text_list]
    token_indices = np.zeros((len(tokens), max_length), dtype=int)
    for i, tweet in enumerate(tokens):
        for j, word in enumerate(tweet):
            if word in word2index:
                token_indices[i, j] = word2index[word]
    return torch.tensor(token_indices)

wrapped_model = CNNBaselineWrapper(model)
class_names = ['hate_speech', 'offensive_language', 'neither']

# Explainability with SHAP
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def shap_analysis(text, wrapped_model, class_names):
    # Define a masker
    masker = shap.maskers.Text(tokenizer)

    # Initialize explainer
    explainer = shap.Explainer(wrapped_model, masker=masker)

    # Explain prediction
    shap_values = explainer([text])

    return shap_values

df['tweet'] = df['tweet'].apply(lambda x: ' '.join(x))
text_to_explain = random.choice(df['tweet'])
print("Text to explain:", text_to_explain)
shap_results = shap_analysis(text_to_explain, wrapped_model, class_names)

# Get predicted label for the text
predicted_label = np.argmax(wrapped_model([text_to_explain])[0])

shap.plots.text(shap_results[0])

def calculate_doe(shap_values, predicted_label):
    feature_scores = [abs(value) for value in shap_values.values[0][:, predicted_label]]
    std_dev = np.std(feature_scores)
    significant_features = len([score for score in feature_scores if score > std_dev])
    return significant_features / len(feature_scores)

doe = calculate_doe(shap_results, predicted_label)
print("Degree of Explainability (DoE):", doe)

# Define number of samples for analysis
num_sam = 20

# Load dataset
def load_custom_dataset(path):
    df = pd.read_csv(path)
    df = df.dropna(subset=['tweet', 'class'])
    return df

# SHAP Analysis
def shap_analysis(text, wrapped_model, class_names):
    # Define a masker
    masker = shap.maskers.Text(tokenizer)

    # Initialize explainer
    explainer = shap.Explainer(wrapped_model, masker=masker)

    # Explain prediction
    shap_values = explainer([text])

    return shap_values

# Calculate Degree of Explainability (DoE)
def calculate_doe(shap_values, predicted_label):
    feature_scores = [abs(value) for value in shap_values.values[0][:, predicted_label]]
    std_dev = np.std(feature_scores)
    significant_features = len([score for score in feature_scores if score > std_dev])
    return significant_features / len(feature_scores)

# Calculate average DoE for multiple samples
def calculate_average_doe(df, wrapped_model, class_names, samples=num_sam):
    doe_values = []
    sample_texts = random.sample(list(df['tweet']), samples)
    for text in sample_texts:
        shap_results = shap_analysis(text, wrapped_model, class_names)
        predicted_label = np.argmax(wrapped_model([text])[0])
        doe = calculate_doe(shap_results, predicted_label)
        doe_values.append(doe)
    average_doe = np.mean(doe_values)
    return average_doe

# Path to the dataset
test_data_path = "./Data/KaggleData.csv"
df = load_custom_dataset(test_data_path)

# Assuming wrapped_model and class_names are defined elsewhere
class_names = ['Hate speech', 'Offensive language', 'Neutral']

# Calculate and print average DoE
average_doe = calculate_average_doe(df, wrapped_model, class_names)
print(f"Average Degree of Explainability (DoE) for {num_sam} samples:", average_doe)